In [6]:
import preprocessData
import calculate
from calculate import Math
from database import bondLengthDatabase, segmentVolumeDatabase
import numpy as np
import sys
import os
import csv
import SGmodel
import resultData

In [48]:
struct = []
f = []
molType = []
allEnergy = []
orgcom = []
spec = []
basises = []
for file in preprocessData.dataName():
    pdgod = preprocessData.getOutputData()
    if file[-3:] == 'out':
        sss = pdgod.dataMolecular(file)
        sss[0], sss[1] = sss[1], sss[0]
        temorg = pdgod.dataOrgcom(file)
        temEnergy = pdgod.dataEnergy(file)
        temStruct = pdgod.dataStructure(file)
        temBasis = pdgod.dataBasis(file)
        temType = pdgod.dataMoltype(file)
        if len(spec) == 0:
            spec.append(sss)
            orgcom.append(temorg)
            allEnergy.append(temEnergy)
            struct.append(temStruct)
            basises.append(temBasis)
            f.append(file)
            molType.append(temType)
        else:
            for i, spe in enumerate(spec):
                if sss[0][2]+sss[1][2] <= spe[0][2]+spe[1][2]:
                    spec.insert(i, sss)
                    orgcom.insert(i, temorg)
                    allEnergy.insert(i, temEnergy)
                    struct.insert(i, temStruct)
                    basises.insert(i, temBasis)
                    f.insert(i, file)
                    molType.insert(i, temType)
                    break       
                try:
                    if sss[0][2]+sss[1][2] > spe[0][2]+spe[1][2] and sss[0][2]+sss[1][2] <= spec[i+1][0][2]+spec[i+1][1][2]:
                        spec.insert(i+1, sss)
                        orgcom.insert(i+1, temorg)
                        allEnergy.insert(i+1, temEnergy)
                        struct.insert(i+1, temStruct)
                        basises.insert(i+1, temBasis)
                        f.insert(i+1, file)
                        molType.insert(i+1, temType)
                        break
                except:pass
                if i == len(spec)-1:
                    spec.append(sss)
                    orgcom.append(temorg)
                    allEnergy.append(temEnergy)
                    struct.append(temStruct)
                    basises.append(temBasis)
                    f.append(file)
                    molType.append(temType)
                    break
    else:continue
print(np.array(f))
print(np.array(orgcom))
print(np.array(spec))


['formicacid-formamide_jtz.out' 'formicacid-acetamide_jtz.out'
 'aceticacid-formamide_jtz.out' 'propanoicacid-formamide_jtz.out'
 'formicacid-propanamide_jtz.out' 'aceticacid-acetamide_jtz.out'
 'aceticacid-acetamide_105.out' 'propanoicacid-acetamide_jtz.out'
 'aceticacid-propanamide_jtz.out' 'propanoicacid-propanamide_jtz.out']
[['Acid' 'Amide']
 ['Acid' 'Amide']
 ['Acid' 'Amide']
 ['Acid' 'Amide']
 ['Acid' 'Amide']
 ['Acid' 'Amide']
 ['Acid' 'Amide']
 ['Acid' 'Amide']
 ['Acid' 'Amide']
 ['Acid' 'Amide']]
[[[1 1 1 3]
  [2 0 1 2]]

 [[1 1 2 5]
  [2 0 1 2]]

 [[1 1 1 3]
  [2 0 2 4]]

 [[1 1 1 3]
  [2 0 3 6]]

 [[1 1 3 7]
  [2 0 1 2]]

 [[1 1 2 5]
  [2 0 2 4]]

 [[1 1 2 5]
  [2 0 2 4]]

 [[1 1 2 5]
  [2 0 3 6]]

 [[1 1 3 7]
  [2 0 2 4]]

 [[1 1 3 7]
  [2 0 3 6]]]


In [11]:
volumn_refDisp_3D, component3D = [], []
for i, fn in enumerate(f):
    print(fn)
    s3 = calculate.Sort.fragmentSort(struct[i])
    frag2d, fragStruct3d = [], []
    for s2 in s3:   
        frag, fragStruct = calculate.Convert.fragmentConvert(s2)
        frag2d.append(frag)
        fragStruct3d.append(fragStruct)
    frag2d, fragStruct3d = calculate.Sort.dicectionSort(frag2d, fragStruct3d)
    print(frag2d)
    print(fragStruct3d)
    model_disp = SGmodel.dispersion(frag2d, fragStruct3d, basises[i])
    vr, component = model_disp.energyDisp(Rrange=6, train_mode=True
                                         , referenceDisp=allEnergy[i][-2])
    volumn_refDisp_3D.append(vr)
    component3D.append(component)

    if molType[i] == 'Dimer':
        name = fn[:-8] + '-dimer'
    else:
        name = fn[:-8]
print(np.array(volumn_refDisp_3D), '\n', component3D)
model_disp.modeling(volumn_refDisp_3D, component3D, maxerr=1)

ethane-acetaldehyde_jtz.out
[['A', 'F'], ['A', 'A']]
[[[1.613367, 1.127693, 0.06799], [1.3775914285714284, -0.8202274285714285, -0.15567714285714285]], [[-1.982254, 0.343806, -0.68602], [-1.9531850000000002, -0.245223, 0.7193900000000001]]]
['A', 'F'] 
 ['A', 'A']
11.5810350741116 4.705035074111599
propane-acetaldehyde_jtz.out
[['A', 'F'], ['A', 'B', 'A']]
[[[-2.212607, -0.773679, 0.54193], [-1.5658034285714284, 0.8923218571428572, -0.29919142857142855]], [[1.35452, -1.259396, -0.6119], [1.603982, -0.331976, 0.57129], [2.122793, 1.02859, 0.12396]]]
['A', 'F'] 
 ['A', 'B', 'A']
11.5810350741116 8.9660701482232
ethane-propanal_jtz.out
[['F', 'B', 'A'], ['A', 'A']]
[[[-0.9017031428571428, 1.219973857142857, -0.27831285714285714], [-1.6900520000000003, -0.356128, 0.62197], [-1.475411, -1.3378430000000001, -0.51563]], [[2.113467, -0.184589, 0.8588299999999999], [2.45539, -0.147475, -0.62597]]]
['F', 'B', 'A'] 
 ['A', 'A']
15.8420701482232 4.705035074111599
propane-propanal_jtz.out
[['F', 'B

['F', 7.381678414711095, -3.04753456, 0.85]

In [50]:
for i, fn in enumerate(f):
    print(fn)
    s3 = calculate.Sort.fragmentSort(struct[i])
    frag2d, fragStruct3d = [], []
    for s2 in s3:   
        frag, fragStruct = calculate.Convert.fragmentConvert(s2)
        frag2d.append(frag)
        fragStruct3d.append(fragStruct)
    frag2d, fragStruct3d = calculate.Sort.dicectionSort(frag2d, fragStruct3d)
    print(frag2d)
    print(fragStruct3d)
    model_disp = SGmodel.dispersion(frag2d, fragStruct3d, basises[i])
    predDisp = model_disp.run(Rrange=7)
    
    print(predDisp)
    print(allEnergy[i][-2])
    if molType[i] == 'Dimer':
        name = fn[:-8] + '-dimer'
    else:
        name = fn[:-8]
    resultData.saveResult(name, 'disp', basises[i], refEnergy=allEnergy[i][-2], predEnergy=predDisp)

formicacid-formamide_jtz.out
[['G'], ['H']]
[[[-1.5586260909090905, 0.0795304545454545, -5.0909090909090914e-05]], [[1.522935285714286, -0.16734404761904761, 3.523809523809524e-05]]]
['G'] 
 ['H']
-7.237297579902722
-7.32141757
formicacid-acetamide_jtz.out
[['G'], ['A', 'H']]
[[[2.0814821818181817, -0.00401063636363636, -3.9090909090909085e-05]], [[-2.94688, -0.18824, -0.00013], [-1.0076879047619047, -0.01129585714285717, 0.00012047619047619049]]]
['G'] 
 ['A', 'H']
-7.532470156445821
-7.68571069
aceticacid-formamide_jtz.out
[['G', 'A'], ['H']]
[[[-1.036724909090909, 0.03028681818181815, 1.9090909090909022e-05], [-2.9658129999999994, 0.074805, -0.00037999999999999997]], [[2.056221285714286, -0.14251014285714286, 0.0003690476190476191]]]
['G', 'A'] 
 ['H']
-7.392090811399803
-7.38570885
propanoicacid-formamide_jtz.out
[['A', 'B', 'G'], ['H']]
[[[-3.288775, 0.747734, -0.00014], [-2.46069, -0.524493, -0.00015], [-0.5435584545454546, -0.2625098181818182, 0.00018090909090909092]], [[2.53885